In [ ]:
%pip install mysql-connector-python

In [3]:
import mysql.connector
import pandas as pd

In [4]:
# creating connection between python and mysql
connection=mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    database="secure_check"
)
cursor=connection.cursor()

In [ ]:
# creating a database schema
query="create database secure_check"
cursor.execute(query)

In [ ]:
# show databases
query=("show databases") 
cursor.execute(query)
for db in cursor:
    print(db)

In [69]:
#Creating a Table police_logs
query="""create table police_logs(
Stop_Date DATE,
Stop_time TIME, Country_name VARCHAR(50),
Driver_gender VARCHAR(20),Driver_age_raw INT,
Driver_age INT,Driver_race VARCHAR(50),
Violation_raw VARCHAR(50), Violation VARCHAR(50),
Search_conducted BOOLEAN, Search_type VARCHAR(100),
Stop_outcome VARCHAR(50), is_arrested BOOLEAN,
Stop_duration VARCHAR(50), Drugs_related_stop BOOLEAN,
Vehicle_Number VARCHAR(100))"""
cursor.execute(query)


In [ ]:
#drop the table 
query="DROP table police_logs"
cursor.execute(query)

In [70]:
# drop columns that are not used
query = """
ALTER TABLE police_logs
DROP COLUMN Driver_age_raw,
DROP COLUMN Violation_raw;
"""
cursor.execute(query)


In [71]:
connection.commit()

In [ ]:
# To view the columns in the table
query="describe police_logs"
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
# Cleaning the dataset before inserting Value from csv to sql using python

#read the csv file
df=pd.read_csv(r"C:\DataScience_project1_Policelogs\traffic_stops - traffic_stops_with_vehicle_number.csv")


In [74]:
#Remove the columns that only contains missing value
df.dropna(axis=1,how='all',inplace=True)

In [ ]:
# Check the sum of NAN values in the column
df.isnull().sum()

In [ ]:
#Handle the NAN value
df['search_type'] = df['search_type'].fillna('Unknown')
df

In [77]:
# Drop Unnecessary Columns
df.drop(['driver_age_raw', 'violation_raw'], axis=1, inplace=True)

In [ ]:
#Check for cleaned DataFrame
df.info()

In [ ]:
# Inserting Pandas dataframe into Mysql table

insert_query = """
INSERT INTO police_logs (
    Stop_Date, Stop_time, Country_name, Driver_gender, Driver_age,
    Driver_race, Violation, Search_conducted, Search_type,
    Stop_outcome, is_arrested, Stop_duration, Drugs_related_stop,
    Vehicle_Number
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(insert_query, df.values.tolist())
connection.commit()
print("Data inserted using executemany()")


In [82]:
# To view the records inside the table police_logs
query="select count(*) from police_logs"
cursor.execute(query)
for data in cursor:
    print(data)

(65538,)


SQL QUERIES MEDIUM LEVEL 

In [ ]:
# Vehicle based

#What are the top 10 vehicles involved in drug-related stops?

query="""SELECT Vehicle_Number, COUNT(*) AS drug_stop_count
FROM police_logs
WHERE drugs_related_stop = TRUE
GROUP BY Vehicle_Number
ORDER BY drug_stop_count DESC
LIMIT 10;"""
cursor.execute(query)
for data in cursor:
    print(data)



In [ ]:
#Which vehicles were most frequently searched?
query="""SELECT Vehicle_Number,count(*) as search_count
FROM police_logs
WHERE search_conducted = TRUE
GROUP BY Vehicle_Number
ORDER BY search_count DESC
"""
cursor.execute(query)
for data in cursor:
    print(data) 

#Demographic-Based


In [11]:
#Which driver age group had the highest arrest rate?
query="""SELECT CASE WHEN driver_age < 18 THEN '<18'
WHEN driver_age BETWEEN 18 AND 25 THEN '18-25'
WHEN driver_age BETWEEN 26 AND 35 THEN '26-35'
WHEN driver_age BETWEEN 36 AND 45 THEN '36-45'
WHEN driver_age BETWEEN 46 AND 55 THEN '46-55'
WHEN driver_age BETWEEN 56 AND 65 THEN '56-65'
ELSE '65+'
END AS driver_age_group,
count(*) as total_stops,
sum(CASE when is_arrested= TRUE then 1 else 0 END) AS TOTAL_ARREST,
Round(sum(CASE when is_arrested= TRUE then 1 else 0 END)*100/count(*),4) AS ARREST_RATE
FROM police_logs
group by driver_age_group
order by arrest_rate desc
limit 1""";
cursor.execute(query)
for data in cursor:
    print(data)


('18-25', 8137, Decimal('4171'), Decimal('51.2597'))


In [105]:
#What is the gender distribution of drivers stopped in each country?
query="""SELECT country_name, driver_gender, COUNT(*) AS total_stop_count
FROM police_logs
GROUP BY country_name, driver_gender
ORDER BY country_name, driver_gender"""
cursor.execute(query)
for data in cursor:
    print(data)

('Canada', 'F', 10997)
('Canada', 'M', 10911)
('India', 'F', 11043)
('India', 'M', 10955)
('USA', 'F', 10841)
('USA', 'M', 10791)


In [146]:
#Which race and gender combination has the highest search rate?

query="""SELECT driver_race, driver_gender,COUNT(*) AS stop_count,
SUM(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END) AS total_searches,
round(100*(SUM(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END)/ COUNT(*)),2) AS search_rate
FROM police_logs GROUP BY driver_race, driver_gender ORDER BY search_rate DESC LIMIT 1 """
cursor.execute(query)
for data in cursor:
    print(data)

('Asian', 'F', 6649, Decimal('3410'), Decimal('51.29'))


Time & Duration Based

In [154]:
#What time of day sees the most traffic stops?
query="""SELECT hour(stop_time) as Hour_of_the_day, count(*) as total_stops,
CASE WHEN HOUR(Stop_time) BETWEEN 5 AND 11 THEN 'Morning'
WHEN HOUR(Stop_time) BETWEEN 12 AND 15 THEN 'Afternoon'
WHEN HOUR(Stop_time) BETWEEN 16 AND 19 THEN 'Evening'
ELSE 'Night'
END AS time_of_day
from police_logs
GROUP BY hour(stop_time),time_of_day
ORDER BY hour(stop_time) desc
LIMIT 1
"""
cursor.execute(query)
for data in cursor:
    print(data)


(23, 2700, 'Night')


In [191]:
#What is the average stop duration for different violations?

query="""SELECT VIOLATION,
avg(CASE WHEN STOP_DURATION="0-15 Min" THEN 7.5 
WHEN STOP_DURATION="16-30 Min" THEN 23
WHEN STOP_DURATION="30+ Min" THEN 35
END) AS avg_duration,COUNT(*) AS TOTAL_STOPS
FROM POLICE_LOGS
GROUP BY VIOLATION
ORDER BY violation
"""
cursor.execute(query)
for data in cursor:
    print(data)


('DUI', Decimal('21.74134'), 13075)
('Other', Decimal('22.20331'), 13194)
('Seatbelt', Decimal('21.83321'), 13007)
('Signal', Decimal('21.71210'), 13112)
('Speeding', Decimal('21.79449'), 13150)


In [ ]:
#Are stops during the night more likely to lead to arrests?
query= """SELECT CASE 
WHEN HOUR(STOP_TIME) BETWEEN 5 AND 11 THEN 'MORNING'
WHEN HOUR(STOP_TIME) BETWEEN 12 AND 3 THEN 'AFTERNOON'
WHEN HOUR(STOP_TIME) BETWEEN 4 AND 7 THEN 'EVENING'
ELSE 'NIGHT' END AS TIME_OF_THE_DAY,
COUNT(*) AS TOTAL_STOPS,
SUM(CASE WHEN IS_ARRESTED=TRUE THEN 1 ELSE 0 END) AS TOTAL_ARREST,
ROUND(SUM(CASE WHEN IS_ARRESTED=TRUE THEN 1 ELSE 0 END)*100/COUNT(*),2) AS ARREST_RATE
FROM POLICE_LOGS
GROUP BY TIME_OF_THE_DAY
ORDER BY ARREST_RATE DESC
"""
cursor.execute(query)
for data in cursor:
    print(data)

('MORNING', 19320, Decimal('9814'), Decimal('50.80'))
('NIGHT', 43458, Decimal('21672'), Decimal('49.87'))
('EVENING', 2760, Decimal('1360'), Decimal('49.28'))


Violation-Based


In [257]:
#Which violations are most associated with searches or arrests?

query="""SELECT VIOLATION,COUNT(*) AS TOTAL_STOPS,
SUM(CASE WHEN SEARCH_CONDUCTED= TRUE THEN 1 ELSE 0 END) AS TOTAL_SEARCH,
SUM(CASE WHEN IS_ARRESTED= TRUE THEN 1 ELSE 0 END) AS TOTAL_ARREST
FROM POLICE_LOGS
GROUP BY VIOLATION
ORDER BY TOTAL_SEARCH DESC,TOTAL_ARREST DESC
"""
cursor.execute(query)
for data in cursor:
    print(data)

('Other', 13194, Decimal('6576'), Decimal('6516'))
('Speeding', 13150, Decimal('6573'), Decimal('6598'))
('Seatbelt', 13007, Decimal('6562'), Decimal('6573'))
('Signal', 13112, Decimal('6509'), Decimal('6549'))
('DUI', 13075, Decimal('6481'), Decimal('6610'))


In [258]:
query="""SELECT VIOLATION,COUNT(*) AS TOTAL_STOPS,
SUM(CASE WHEN SEARCH_CONDUCTED= TRUE THEN 1 ELSE 0 END) AS TOTAL_SEARCH,
SUM(CASE WHEN IS_ARRESTED= TRUE THEN 1 ELSE 0 END) AS TOTAL_ARREST
FROM POLICE_LOGS
GROUP BY VIOLATION
ORDER BY TOTAL_ARREST DESC,TOTAL_SEARCH DESC
"""
cursor.execute(query)
for data in cursor:
    print(data)

('DUI', 13075, Decimal('6481'), Decimal('6610'))
('Speeding', 13150, Decimal('6573'), Decimal('6598'))
('Seatbelt', 13007, Decimal('6562'), Decimal('6573'))
('Signal', 13112, Decimal('6509'), Decimal('6549'))
('Other', 13194, Decimal('6576'), Decimal('6516'))


In [256]:
#Which violations are most common among younger drivers (<25)?
query="""SELECT VIOLATION,COUNT(*) AS YOUNG_DRIVERS_COUNT
FROM POLICE_LOGS
WHERE DRIVER_AGE<25
GROUP BY VIOLATION
ORDER BY YOUNG_DRIVERS_COUNT DESC
LIMIT 1
"""
cursor.execute(query)
for data in cursor:
    print(data)

('Speeding', 1476)


In [267]:
#Is there a violation that rarely results in search or arrest?
query="""SELECT violation,COUNT(*) AS total_stops,
SUM(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END) AS total_searches,
SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) AS total_arrests,
ROUND(100.0 * (SUM(CASE WHEN search_conducted = TRUE THEN 1 ELSE 0 END) + 
                   SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END)) / COUNT(*), 2) AS combined_rate_percent
FROM police_logs
GROUP BY violation
ORDER BY combined_rate_percent ASC
LIMIT 1
"""
cursor.execute(query)
for data in cursor:
    print(data)

('Other', 13194, Decimal('6576'), Decimal('6516'), Decimal('99.23'))


Location-Based


In [272]:
#Which countries report the highest rate of drug-related stops?
query="""select country_name,count(*) as total_stops,
Round(100*sum(case when drugs_related_stop=TRUE then 1 else 0 end)/count(*),2) as drugs_stop_percentage
from police_logs
group by country_name
order by drugs_stop_percentage desc
limit 1
"""
cursor.execute(query)
for data in cursor:
    print(data)

('USA', 21632, Decimal('50.37'))


In [274]:
#What is the arrest rate by country and violation?
query="""SELECT
ROUND(SUM(CASE WHEN IS_ARRESTED= TRUE THEN 1 ELSE 0 END)*100/COUNT(*),2) AS ARREST_RATE,
COUNTRY_NAME,VIOLATION,COUNT(*) AS TOTAL_STOPS
FROM POLICE_LOGS
GROUP BY COUNTRY_NAME,VIOLATION
ORDER BY ARREST_RATE DESC
"""
cursor.execute(query)
for data in cursor:
    print(data)

(Decimal('51.13'), 'Canada', 'DUI', 4414)
(Decimal('50.84'), 'India', 'DUI', 4420)
(Decimal('50.76'), 'India', 'Speeding', 4358)
(Decimal('50.65'), 'India', 'Signal', 4444)
(Decimal('50.58'), 'USA', 'Seatbelt', 4241)
(Decimal('50.56'), 'India', 'Seatbelt', 4345)
(Decimal('50.46'), 'Canada', 'Seatbelt', 4421)
(Decimal('50.29'), 'Canada', 'Other', 4371)
(Decimal('49.99'), 'USA', 'Signal', 4341)
(Decimal('49.94'), 'Canada', 'Speeding', 4375)
(Decimal('49.83'), 'USA', 'Speeding', 4417)
(Decimal('49.66'), 'USA', 'DUI', 4241)
(Decimal('49.29'), 'India', 'Other', 4431)
(Decimal('49.18'), 'Canada', 'Signal', 4327)
(Decimal('48.59'), 'USA', 'Other', 4392)


In [283]:
#Which country has the most stops with search conducted?
query="""SELECT 
COUNTRY_NAME,COUNT(*) AS TOTAL_SEARCH_CONDUCTED_STOPS
FROM POLICE_LOGS
WHERE SEARCH_CONDUCTED=TRUE
GROUP BY COUNTRY_NAME
ORDER BY TOTAL_SEARCH_CONDUCTED_STOPS DESC
LIMIT 1
"""
cursor.execute(query)
for data in cursor:
    print(data)

('Canada', 11020)


Complex Queries

In [10]:
#Yearly Breakdown of Stops and Arrests by Country (Using Subquery and Window Functions)
query="""SELECT COUNTRY_NAME,YEAR,TOTAL_STOPS,TOTAL_ARREST,
ROUND(100*TOTAL_ARREST/TOTAL_STOPS,2) AS ARREST_RATE,
RANK() OVER(PARTITION BY COUNTRY_NAME ORDER BY YEAR) AS YEAR_RANK
FROM
(SELECT COUNTRY_NAME,YEAR(STOP_DATE) AS YEAR,
COUNT(*) AS TOTAL_STOPS,
SUM(CASE WHEN IS_ARRESTED= TRUE THEN 1 ELSE 0 END) AS TOTAL_ARREST
FROM POLICE_LOGS 
GROUP BY COUNTRY_NAME,YEAR) AS YEAR_DATA
"""
cursor.execute(query)
for data in cursor:
    print(data)

('Canada', 2020, 21908, Decimal('10999'), Decimal('50.21'), 1)
('India', 2020, 21998, Decimal('11091'), Decimal('50.42'), 1)
('USA', 2020, 21632, Decimal('10756'), Decimal('49.72'), 1)


In [ ]:
#Driver Violation Trends Based on Age and Race (Join with Subquery)
QUERY = """
WITH age_grouped AS (
SELECT violation,driver_race,
CASE WHEN driver_age < 18 THEN 'Under 18'
WHEN driver_age BETWEEN 18 AND 25 THEN '18-25'
WHEN driver_age BETWEEN 26 AND 40 THEN '26-40'
WHEN driver_age BETWEEN 41 AND 60 THEN '41-60'
ELSE '60+' 
END AS age_group
FROM police_logs
)
SELECT 
ag.age_group,
ag.driver_race,
ag.violation,
COUNT(*) AS total_violations
FROM age_grouped ag
GROUP BY ag.age_group, ag.driver_race, ag.violation
ORDER BY ag.age_group, total_violations DESC;
"""
cursor.execute(QUERY)
for row in cursor:
    print(row)

('18-25', 'Black', 'Speeding', 365)
('18-25', 'Black', 'Signal', 355)
('18-25', 'Other', 'Seatbelt', 349)
('18-25', 'Asian', 'Speeding', 348)
('18-25', 'White', 'Signal', 347)
('18-25', 'Hispanic', 'Other', 342)
('18-25', 'White', 'Speeding', 334)
('18-25', 'Hispanic', 'DUI', 333)
('18-25', 'Black', 'Other', 332)
('18-25', 'Other', 'Speeding', 332)
('18-25', 'Other', 'Signal', 330)
('18-25', 'Asian', 'Seatbelt', 330)
('18-25', 'Other', 'Other', 325)
('18-25', 'White', 'Other', 321)
('18-25', 'White', 'Seatbelt', 316)
('18-25', 'Black', 'DUI', 313)
('18-25', 'Other', 'DUI', 312)
('18-25', 'Hispanic', 'Seatbelt', 312)
('18-25', 'Hispanic', 'Speeding', 311)
('18-25', 'Asian', 'Other', 310)
('18-25', 'White', 'DUI', 309)
('18-25', 'Hispanic', 'Signal', 307)
('18-25', 'Black', 'Seatbelt', 305)
('18-25', 'Asian', 'DUI', 302)
('18-25', 'Asian', 'Signal', 297)
('26-40', 'Asian', 'Signal', 662)
('26-40', 'Asian', 'Speeding', 652)
('26-40', 'Hispanic', 'Seatbelt', 642)
('26-40', 'Asian', 'DUI', 

In [298]:
#Time Period Analysis of Stops (Joining with Date Functions) , Number of Stops by Year,Month, Hour of the Day
QUERY="""SELECT YEAR(STOP_DATE) AS YEAR, MONTH(STOP_DATE) AS MONTH, HOUR(STOP_TIME) AS HOUR,
COUNT(*) AS TOTAL_STOPS
FROM POLICE_LOGS
GROUP BY YEAR,MONTH,HOUR
ORDER BY YEAR,MONTH,HOUR
"""
cursor.execute(QUERY)
for data in cursor:
    print(data)

(2020, 1, 0, 1860)
(2020, 1, 1, 1860)
(2020, 1, 2, 1860)
(2020, 1, 3, 1860)
(2020, 1, 4, 1860)
(2020, 1, 5, 1860)
(2020, 1, 6, 1860)
(2020, 1, 7, 1860)
(2020, 1, 8, 1860)
(2020, 1, 9, 1860)
(2020, 1, 10, 1860)
(2020, 1, 11, 1860)
(2020, 1, 12, 1860)
(2020, 1, 13, 1860)
(2020, 1, 14, 1860)
(2020, 1, 15, 1860)
(2020, 1, 16, 1860)
(2020, 1, 17, 1860)
(2020, 1, 18, 1860)
(2020, 1, 19, 1860)
(2020, 1, 20, 1860)
(2020, 1, 21, 1860)
(2020, 1, 22, 1860)
(2020, 1, 23, 1860)
(2020, 2, 0, 900)
(2020, 2, 1, 900)
(2020, 2, 2, 900)
(2020, 2, 3, 900)
(2020, 2, 4, 900)
(2020, 2, 5, 900)
(2020, 2, 6, 900)
(2020, 2, 7, 900)
(2020, 2, 8, 900)
(2020, 2, 9, 900)
(2020, 2, 10, 900)
(2020, 2, 11, 900)
(2020, 2, 12, 858)
(2020, 2, 13, 840)
(2020, 2, 14, 840)
(2020, 2, 15, 840)
(2020, 2, 16, 840)
(2020, 2, 17, 840)
(2020, 2, 18, 840)
(2020, 2, 19, 840)
(2020, 2, 20, 840)
(2020, 2, 21, 840)
(2020, 2, 22, 840)
(2020, 2, 23, 840)


In [304]:
#Violations with High Search and Arrest Rates (Window Function)
QUERY="""SELECT VIOLATION, COUNT(*) AS TOTAL_STOP,
SUM(CASE WHEN SEARCH_CONDUCTED=TRUE THEN 1 ELSE 0 END) AS TOTAL_SEARCH,
ROUND(SUM(CASE WHEN SEARCH_CONDUCTED=TRUE THEN 1 ELSE 0 END)*100/COUNT(*),2) AS TOTAL_SEARCH_RATE,
RANK() OVER (ORDER BY 1.0 * SUM(CASE WHEN SEARCH_CONDUCTED=TRUE THEN 1 ELSE 0 END)/COUNT(*) DESC) AS search_rank,
SUM(CASE WHEN IS_ARRESTED=TRUE THEN 1 ELSE 0 END) AS TOTAL_ARREST,
ROUND(SUM(CASE WHEN IS_ARRESTED=TRUE THEN 1 ELSE 0 END)*100/COUNT(*),2) AS TOTAL_ARREST_RATE,
RANK() OVER (ORDER BY 1.0 * SUM(CASE WHEN is_arrested = TRUE THEN 1 ELSE 0 END) / COUNT(*) DESC) AS arrest_rank
FROM POLICE_LOGS
GROUP BY VIOLATION
ORDER BY TOTAL_SEARCH_RATE DESC,TOTAL_ARREST_RATE DESC
"""
cursor.execute(QUERY)
for data in cursor:
    print(data)


('Seatbelt', 13007, Decimal('6562'), Decimal('50.45'), 1, Decimal('6573'), Decimal('50.53'), 2)
('Speeding', 13150, Decimal('6573'), Decimal('49.98'), 2, Decimal('6598'), Decimal('50.17'), 3)
('Other', 13194, Decimal('6576'), Decimal('49.84'), 3, Decimal('6516'), Decimal('49.39'), 5)
('Signal', 13112, Decimal('6509'), Decimal('49.64'), 4, Decimal('6549'), Decimal('49.95'), 4)
('DUI', 13075, Decimal('6481'), Decimal('49.57'), 5, Decimal('6610'), Decimal('50.55'), 1)


In [8]:
#Driver Demographics by Country (Age, Gender, and Race)
QUERY="""SELECT 
    country_name,
    ROUND(AVG(driver_age), 1) AS avg_age,
    SUM(CASE WHEN driver_gender = 'M' THEN 1 ELSE 0 END) AS male_drivers,
    SUM(CASE WHEN driver_gender = 'F' THEN 1 ELSE 0 END) AS female_drivers,
    COUNT(*) AS total_drivers,
    SUM(CASE WHEN driver_race = 'White' THEN 1 ELSE 0 END) AS white_drivers,
    SUM(CASE WHEN driver_race = 'Black' THEN 1 ELSE 0 END) AS black_drivers,
    SUM(CASE WHEN driver_race = 'Hispanic' THEN 1 ELSE 0 END) AS hispanic_drivers,
    SUM(CASE WHEN driver_race = 'Asian' THEN 1 ELSE 0 END) AS asian_drivers,
    SUM(CASE WHEN driver_race NOT IN ('White','Black','Hispanic','Asian') THEN 1 ELSE 0 END) AS other_race
FROM police_logs
GROUP BY country_name
ORDER BY country_name
"""
cursor.execute(QUERY)
for data in cursor:
    print(data)


('Canada', Decimal('49.1'), Decimal('10911'), Decimal('10997'), 21908, Decimal('4429'), Decimal('4342'), Decimal('4395'), Decimal('4344'), Decimal('4398'))
('India', Decimal('49.1'), Decimal('10955'), Decimal('11043'), 21998, Decimal('4346'), Decimal('4418'), Decimal('4406'), Decimal('4451'), Decimal('4377'))
('USA', Decimal('49.1'), Decimal('10791'), Decimal('10841'), 21632, Decimal('4393'), Decimal('4371'), Decimal('4205'), Decimal('4316'), Decimal('4347'))


In [305]:
#Top 5 Violations with Highest Arrest Rates
QUERY="""SELECT VIOLATION,COUNT(*) AS TOTAL_STOPS,
round(100*sum(case when IS_ARRESTED=TRUE then 1 else 0 end)/COUNT(*),2) AS ARREST_RATE
FROM POLICE_LOGS
GROUP BY VIOLATION
ORDER BY ARREST_RATE DESC
LIMIT 5
"""
cursor.execute(QUERY)
for data in cursor:
    print(data)

('DUI', 13075, Decimal('50.55'))
('Seatbelt', 13007, Decimal('50.53'))
('Speeding', 13150, Decimal('50.17'))
('Signal', 13112, Decimal('49.95'))
('Other', 13194, Decimal('49.39'))
